In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Evaluation.K_FoldEvaluator import K_FoldEvaluator
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()

URM_train_list=[]
URM_validation_list=[]
evaluator_list=[]

In [3]:
for k in range(5):
    URM_train,URM_validation = split_train_in_two_percentage_global_sample(URM, train_percentage=0.8)
    URM_train_list.append(URM_train)
    URM_validation_list.append(URM_validation)
    evaluator_list.append(EvaluatorHoldout(URM_validation, cutoff_list=[10],verbose=False))

In [4]:
URM_combined_list=[]
for URM_i in URM_train_list:
    URM_combined_list.append(combine_matrices(ICM=ICM,URM=URM_i))

In [5]:
from bayes_opt import BayesianOptimization

In [6]:
rp3_recommenders=[]
slime_recommenders=[]
IALS_recommenders=[]
similarity_recommenders=[]
recommenders=[]


for i in range(len(URM_train_list)):
    IALS_recommenders.append(IALSRecommender(URM_train=URM_combined_list[i]))
    IALS_recommenders[i].fit(epochs=10, 
                             num_factors=42, 
                             alpha=0.7617528864750021, 
                             reg=8.926401306541349)
    
    rp3_recommenders.append(RP3betaRecommender(URM_train=URM_combined_list[i]))
    rp3_recommenders[i].fit(topK=181, 
                            alpha=0.5709402717259106, 
                            beta=0.3566066452521787, 
                            implicit=True, 
                            normalize_similarity=True)
    
    slime_recommenders.append(MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_combined_list[i]))
    slime_recommenders[i].fit(topK=420, 
                              l1_ratio=0.06061216785568925, 
                              alpha=0.00554982938879437, 
                              workers = 20)
    
    sim_slime = slime_recommenders[i].W_sparse
    sim_rp3 = rp3_recommenders[i].W_sparse
    
    similarity_recommenders.append(ItemKNNSimilarityHybridRecommender(URM_train=URM_combined_list[i],
                                                                     Similarity_1=sim_rp3,
                                                                     Similarity_2=sim_slime))
    similarity_recommenders[i].fit(topK=494, 
                                   alpha=0.1597863887003789)
    
    
    recommender = MergeTwoModelsByScores(URM_train=URM_combined_list[i],
                                         recommenders=[similarity_recommenders[i],IALS_recommenders[i]])
    recommenders.append(recommender)


IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 33.75 sec
IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 39.62 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 45.50 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 52.36 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 58.78 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 1.08 min
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 1.18 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.28 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.38 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.49 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.49 min


100%|████████████████████████████████████▉| 18032/18059 [10:43<00:00, 71.28it/s]

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 7.15 sec
IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 14.15 sec


100%|████████████████████████████████████▉| 18056/18059 [11:00<00:00, 71.28it/s]

IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 20.88 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 27.80 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 34.44 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 41.02 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 47.57 sec
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 54.18 sec
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.01 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.12 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.12 min


...


 23%|████████▋                             | 4136/18059 [02:26<03:59, 58.19it/s]


100%|████████████████████████████████████▉| 18056/18059 [15:34<00:00, 19.31it/s]

 43%|████████████████▌                     | 7848/18059 [04:30<07:51, 21.67it/s]


 62%|███████████████████████              | 11272/18059 [06:16<01:36, 70.58it/s]


 82%|██████████████████████████████▎      | 14824/18059 [08:15<03:21, 16.09it/s]


 99%|████████████████████████████████████▊| 17968/18059 [09:46<00:01, 46.72it/s]

In [7]:
tuning_params = {
    "alpha":(0,1),
    "beta":(0,1)
}

In [8]:

def BO_function(alpha,beta):
    MAP = 0
    k = 5
    cutoff=10
    for i in range(len(recommenders)):
        recommenders[i].fit(alpha,beta)
        result_df, _ = evaluator_list[i].evaluateRecommender(recommenders[i])
        MAP += result_df.loc[cutoff]["MAP"]
        
    return MAP/k


In [11]:
optimizer = BayesianOptimization(f=BO_function,
                                pbounds=tuning_params,
                                verbose=5,
                                random_state=5)

optimizer.maximize(init_points=200,n_iter=200)


|   iter    |  target   |   alpha   |   beta    |
-------------------------------------------------
|  1        |  0.2401   |  0.222    |  0.8707   |
|  2        |  0.2395   |  0.2067   |  0.9186   |
|  3        |  0.2461   |  0.4884   |  0.6117   |
|  4        |  0.2482   |  0.7659   |  0.5184   |
|  5        |  0.2483   |  0.2968   |  0.1877   |
|  6        |  0.2367   |  0.08074  |  0.7384   |
|  7        |  0.2491   |  0.4413   |  0.1583   |
|  8        |  0.2492   |  0.8799   |  0.2741   |
|  9        |  0.248    |  0.4142   |  0.2961   |
|  10       |  0.2471   |  0.6288   |  0.5798   |
|  11       |  0.2489   |  0.5999   |  0.2658   |
|  12       |  0.2473   |  0.2847   |  0.2536   |
|  13       |  0.2489   |  0.3276   |  0.1442   |
|  14       |  0.2383   |  0.1656   |  0.9639   |
|  15       |  0.2491   |  0.9602   |  0.1884   |
|  16       |  0.237    |  0.02431  |  0.2046   |
|  17       |  0.2465   |  0.6998   |  0.7795   |
|  18       |  0.2346   |  0.02293  |  0.5777   |


|  100      |  0.2493   |  0.2456   |  0.07127  |
|  101      |  0.2478   |  0.3477   |  0.01285  |
|  102      |  0.2491   |  0.1656   |  0.06037  |
|  103      |  0.2461   |  0.2782   |  0.3483   |
|  104      |  0.2459   |  0.5912   |  0.775    |
|  105      |  0.2493   |  0.6248   |  0.1597   |
|  106      |  0.2475   |  0.8938   |  0.7306   |
|  107      |  0.246    |  0.4978   |  0.6436   |
|  108      |  0.2491   |  0.5095   |  0.1682   |
|  109      |  0.2451   |  0.6415   |  0.9882   |
|  110      |  0.2492   |  0.7703   |  0.1815   |
|  111      |  0.2373   |  0.1198   |  0.9303   |
|  112      |  0.2475   |  0.9101   |  0.7492   |
|  113      |  0.249    |  0.7605   |  0.1429   |
|  114      |  0.2492   |  0.8119   |  0.1923   |
|  115      |  0.2422   |  0.1547   |  0.4071   |
|  116      |  0.2333   |  0.000382 |  0.8455   |
|  117      |  0.2393   |  0.1611   |  0.7393   |
|  118      |  0.2458   |  0.5829   |  0.7703   |
|  119      |  0.2448   |  0.6      |  0.9615   |


|  200      |  0.2392   |  0.2075   |  0.9717   |
|  201      |  0.2493   |  1.0      |  0.2688   |
|  202      |  0.2472   |  1.0      |  0.0      |
|  203      |  0.2492   |  0.5911   |  0.1282   |
|  204      |  0.2493   |  0.9672   |  0.2875   |
|  205      |  0.2493   |  0.6859   |  0.1831   |
|  206      |  0.2492   |  0.9999   |  0.2237   |
|  207      |  0.2491   |  0.2071   |  0.07877  |
|  208      |  0.2482   |  0.1429   |  0.009637 |
|  209      |  0.2493   |  0.5941   |  0.1763   |
|  210      |  0.2493   |  0.7641   |  0.2285   |
|  211      |  0.2493   |  0.8946   |  0.2193   |
|  212      |  0.2493   |  0.9731   |  0.257    |
|  213      |  0.2494   |  0.6553   |  0.1849   |
|  214      |  0.2492   |  1.0      |  0.3117   |
|  215      |  0.2493   |  0.659    |  0.1625   |
|  216      |  0.2493   |  0.7214   |  0.2096   |
|  217      |  0.2493   |  0.6244   |  0.1885   |
|  218      |  0.2492   |  0.685    |  0.2094   |
|  219      |  0.2493   |  0.7213   |  0.1829   |


|  300      |  0.2494   |  0.985    |  0.2771   |
|  301      |  0.2493   |  0.9712   |  0.2696   |
|  302      |  0.2494   |  0.4524   |  0.1293   |
|  303      |  0.2493   |  0.249    |  0.0659   |
|  304      |  0.2493   |  0.9999   |  0.2826   |
|  305      |  0.2493   |  0.7547   |  0.2051   |
|  306      |  0.2494   |  0.6573   |  0.1879   |
|  307      |  0.2494   |  0.4077   |  0.1129   |
|  308      |  0.2493   |  0.4145   |  0.1126   |
|  309      |  0.2493   |  0.4531   |  0.135    |
|  310      |  0.2473   |  0.6766   |  0.003393 |
|  311      |  0.2489   |  0.8878   |  0.388    |
|  312      |  0.2487   |  0.7458   |  0.3922   |
|  313      |  0.2493   |  0.6536   |  0.1799   |
|  314      |  0.2493   |  0.4006   |  0.1161   |
|  315      |  0.2494   |  0.9923   |  0.2781   |
|  316      |  0.2494   |  0.4441   |  0.1212   |
|  317      |  0.2493   |  0.8139   |  0.2188   |
|  318      |  0.2493   |  0.8071   |  0.2285   |
|  319      |  0.2493   |  0.7066   |  0.1902   |


|  400      |  0.2494   |  0.9838   |  0.2764   |


In [12]:
import json
with open("tuning_KFOLD/RP3hSLIME_IALS/" + recommenders[0].RECOMMENDER_NAME + "_logs.json", 'w') as json_file:
    json.dump(optimizer.max, json_file)